In [136]:
import requests
import json
class Influx():
    def __init__(self):
        self.write_url = 'http://host.docker.internal:8086/api/v2/write'
        self.query_url = 'http://host.docker.internal:8086/api/v2/query'
        self.org = 'test'
        self.bucket = 'test'
        self.precision ='ms'
        self.token = '6b9JljDYHPYro1C3-cv8MrAQtmtUM3WrZlRjkiSEhzZUPYFvETKG0WMVLf6vKd0tcN0awHkul_zTbAuK8IaNTw=='
    
    def write(self, line):
        headers = {'Authorization' : f'Token {self.token}'}
        params = {'org': self.org, 'bucket': self.bucket, 'precision': self.precision}
        return requests.post(self.write_url, headers=headers, params=params, data = line)
    
    def query(self, flux):
        headers = {'Authorization' : f'Token {self.token}'}
        params = {'org': self.org}
        data = {}
        data["type"] = "flux"
        data["query"] = flux
    
        
        return requests.post(self.query_url, headers=headers, params=params, json = data)
    
    
        

In [137]:
influx= Influx()

In [123]:
data = '''disk,created_date=2021-07-14,created_at=2021-07-1405:37:56,tag_id=2575,additional_tags=91 usage_user=85,usage_system=89,usage_idle=19,usage_nice=72,usage_iowait=95,usage_irq=31,usage_softirq=14,usage_steal=42,usage_guest=14,usage_guest_nice=13 1626241076763
disk,created_date=2021-07-14,created_at=2021-07-1405:37:56,tag_id=0363,additional_tags=65 usage_user=17,usage_system=24,usage_idle=86,usage_nice=85,usage_iowait=17,usage_irq=04,usage_softirq=09,usage_steal=74,usage_guest=27,usage_guest_nice=17 1626241076764
disk,created_date=2021-07-14,created_at=2021-07-1405:37:56,tag_id=8351,additional_tags=45 usage_user=04,usage_system=64,usage_idle=65,usage_nice=16,usage_iowait=80,usage_irq=55,usage_softirq=62,usage_steal=94,usage_guest=92,usage_guest_nice=25 1626241076765
disk,created_date=2021-07-14,created_at=2021-07-1405:37:56,tag_id=6968,additional_tags=59 usage_user=18,usage_system=12,usage_idle=75,usage_nice=06,usage_iowait=13,usage_irq=40,usage_softirq=55,usage_steal=40,usage_guest=37,usage_guest_nice=32 1626241076766
disk,created_date=2021-07-14,created_at=2021-07-1405:37:56,tag_id=0970,additional_tags=54 usage_user=96,usage_system=15,usage_idle=64,usage_nice=24,usage_iowait=40,usage_irq=91,usage_softirq=54,usage_steal=83,usage_guest=39,usage_guest_nice=48 1626241076767
disk,created_date=2021-07-14,created_at=2021-07-1405:37:56,tag_id=6058,additional_tags=77 usage_user=42,usage_system=63,usage_idle=93,usage_nice=12,usage_iowait=35,usage_irq=91,usage_softirq=58,usage_steal=19,usage_guest=19,usage_guest_nice=61 1626241076768
disk,created_date=2021-07-14,created_at=2021-07-1405:37:56,tag_id=8529,additional_tags=05 usage_user=75,usage_system=56,usage_idle=98,usage_nice=89,usage_iowait=13,usage_irq=07,usage_softirq=46,usage_steal=38,usage_guest=51,usage_guest_nice=90 1626241076769
disk,created_date=2021-07-14,created_at=2021-07-1405:37:56,tag_id=3669,additional_tags=47 usage_user=36,usage_system=25,usage_idle=57,usage_nice=96,usage_iowait=95,usage_irq=08,usage_softirq=36,usage_steal=01,usage_guest=75,usage_guest_nice=20 1626241076770
disk,created_date=2021-07-14,created_at=2021-07-1405:37:56,tag_id=7780,additional_tags=90 usage_user=18,usage_system=94,usage_idle=42,usage_nice=54,usage_iowait=17,usage_irq=07,usage_softirq=66,usage_steal=84,usage_guest=09,usage_guest_nice=24 1626241076771
disk,created_date=2021-07-14,created_at=2021-07-1405:37:56,tag_id=1237,additional_tags=25 usage_user=38,usage_system=43,usage_idle=87,usage_nice=89,usage_iowait=75,usage_irq=13,usage_softirq=11,usage_steal=98,usage_guest=93,usage_guest_nice=07 1626241076772'''
r = influx.write(data)



In [124]:
print(r.status_code)

204


In [125]:
print(r.text)

In [138]:
query = '''
from(bucket: "test")
  |> range(start: -2d)
  |> group(columns: ["_measurement"])
  |> filter(fn: (r) =>
      r["_measurement"] == "cpu" and
      r["_field"] == "usage_guest"
  )
  |> count()
  |> yield()
'''

In [142]:
r = influx.query(query)
print('r.text', r.text)

result = r.text
result_row = result.split('\n')[1]
result_count = result_row.split(',')[-1]

print('result count', result_count)

r.text ,result,table,_start,_stop,_measurement,_value
,_result,0,2021-07-12T20:50:07.981411Z,2021-07-14T20:50:07.981411Z,cpu,20


result count 20
